In [ ]:
#**Add python version you wish** to list
!sudo apt-get update -y
!sudo apt-get install python3.6
from IPython.display import clear_output
clear_output()
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 1

# Choose one of the given alternatives:
!sudo update-alternatives --config python3

# This one used to work but now NOT(for me)!
# !sudo update-alternatives --config python

# Check the result
!python3 --version

# Attention: Install pip (... needed!)
!sudo apt install python3-pip

There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.9   2         auto mode
  1            /usr/bin/python3.6   1         manual mode
  2            /usr/bin/python3.8   1         manual mode
  3            /usr/bin/python3.9   2         manual mode

Press <enter> to keep the current choice[*], or type selection number: 1
update-alternatives: using /usr/bin/python3.6 to provide /usr/bin/python3 (python3) in manual mode
Python 3.6.15
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python-pip-whl python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-pip python3-setuptools python3-wheel
0 upgraded, 4 newly installed, 0 to rem

In [ ]:
!python3 --version

Python 3.6.15


In [ ]:
!sudo apt install python3-pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pip is already the newest version (20.0.2-5ubuntu1.8).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!sudo apt install python3.6-distutils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python3.6-lib2to3
The following NEW packages will be installed:
  python3.6-distutils python3.6-lib2to3
0 upgraded, 2 newly installed, 0 to remove and 24 not upgraded.
Need to get 308 kB of archives.
After this operation, 1,232 kB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.6-lib2to3 all 3.6.15-1+focal3 [122 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.6-distutils all 3.6.15-1+focal3 [187 kB]
Fetched 308 kB in 2s (172 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debc

In [ ]:
!pip install torch==1.8.1 torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 8.7 kB/s 
  Using cached torchtext-0.6.0-py3-none-any.whl (64 kB)
     |████████████████████████████████| 14.8 MB 66.3 MB/s 
     |████████████████████████████████| 78 kB 230 kB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 1.3 MB 61.0 MB/s 
     |████████████████████████████████| 140 kB 72.3 MB/s 
     |████████████████████████████████| 61 kB 115 kB/s 
     |████████████████████████████████| 155 kB 67.5 MB/s 


In [ ]:
!pip install transformers

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/autocompletion.py", line 9, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/main_parser.py", line 7, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/cmdoptions.py", line 19, in <module>
    from distutils.util import strtobool
ModuleNotFoundError: No module named 'distutils.util'


In [ ]:
import sys
sys.path.insert(0,'/content')

In [ ]:
from functools import partial

from torchtext.datasets import IMDB
# from torchtext.data import Field, LabelField, BucketIterator
from torchtext.data import Field, LabelField, BucketIterator

from transformers import BertTokenizer

# from collators import MLMCollator, SequentialMLMCollator, PermutationMLMCollator, \
#                       TextInfillingCollator, TokenDeletionCollator, DocumentRotationCollator, \
#                       PermutationCollator

ImportError: ignored

In [ ]:

class MultiTaskCollator(MLMCollator):
    def __init__(self, tokenizer, mask_probability, nomask_id=-100, policy=None, special_token_ids=None, masking_schedule=None, nsp_probability=0.5):
        super(MultiTaskCollator, self).__init__(tokenizer, mask_probability, nomask_id, policy, special_token_ids, masking_schedule)
        self.nsp_probability = nsp_probability

    def __call__(self, examples, training_step=None):
        input_ids, labels, nsp_labels = self._preprocess_batch(examples)
        input_ids, mlm_labels = self.mask_tokens(input_ids=input_ids, training_step=training_step)
        return input_ids, mlm_labels, nsp_labels

    def _preprocess_batch(self, examples):
        # Your existing preprocessing logic for MLM
        # ...

        # Next Sentence Prediction preprocessing
        nsp_labels = []
        for i, example in enumerate(examples):
            if torch.rand(1 < self.nsp_probability:
                # Swap current example with the next one
                next_example = examples[(i + 1) % len(examples)]
                example = torch.cat((example[:self.tokenizer.sep_token_id + 1], next_example[self.tokenizer.sep_token_id + 1:]))
                nsp_labels.append(1)
            else:
                nsp_labels.append(0)
        nsp_labels = torch.tensor(nsp_labels, dtype=torch.long)

        return examples, nsp_labels

class MultiTaskModel(nn.Module):
    def __init__(self, base_model, vocab_size):
        super(MultiTaskModel, self).__init__()
        self.base_model = base_model
        self.mlm_head = nn.Linear(base_model.config.hidden_size, vocab_size)
        self.nsp_head = nn.Linear(base_model.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.base_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output, pooled_output = outputs[:2]
        mlm_preds = self.mlm_head(sequence_output)
        nsp_preds = self.nsp_head(pooled_output)
        return mlm_preds, nsp_preds


In [ ]:

import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence


class DataCollator(object):
    def __init__(self):
        pass

    def __call__(self, examples):
        pass

    def _preprocess_batch(self, examples):
        """
        Preprocesses tensors: returns the current examples in case all the tensors are the same
        length, otherwise pads tensors with [PAD] until they are the same length. This function
        is made adaptable to both work with (i) tuple of tensors as is the case when __call__() is
        used as the collate_fn() function of a torch.utils.data.DataLoader() and with (ii) tensor
        with shape (B, ...) where B = batch size (i.e. pre-tensorized)
        """
        if all(x.size(0) == examples[0].size(0) for x in examples):
            if isinstance(examples, tuple):
                return torch.stack(examples, dim=0)
            elif isinstance(examples, torch.Tensor):
                return examples
            else:
                raise ValueError('The type of examples "%s" is not recognized!' % str(type(examples)))
        else:
            if self.tokenizer._pad_token is None:
                raise ValueError('Pad token not found in @tokenizer!')
            elif isinstance(examples, tuple):
                return pad_sequence(examples, batch_first=True, padding_value=self.tokenizer.pad_token_id)
            elif isinstance(examples, torch.Tensor):
                return pad_sequence([example for example in examples], batch_first=True, padding_value=self.tokenizer.pad_token_id)
            else:
                raise ValueError('The type of examples "%s" is not recognized!' % str(type(examples)))



# from transformers import DataCollator
# import torch
# import numpy as np

class MLMCollator(DataCollator):
    def __init__(self, tokenizer, mask_probability, nomask_id=-100, policy=None, special_token_ids=None, masking_schedule=None):
        super(MLMCollator).__init__()
        if tokenizer.mask_token is None:
            raise ValueError(' Tokenizer is missing the mask token (e.g. [MASK], <mask>, etc.)')
        self.tokenizer = tokenizer
        self.mask_probability = mask_probability
        self.mask_id = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)
        self.nomask_id = nomask_id
        self.policy = policy or [0.8, 0.1, 0.1]
        if sum(self.policy) != 1.0:
            raise ValueError('The elements of @policy should sum up to 1.')
        self.special_token_ids = special_token_ids or [self.tokenizer.bos_token_id,
                                                        self.tokenizer.eos_token_id,
                                                        self.tokenizer.sep_token_id,
                                                        self.tokenizer.pad_token_id,
                                                        self.tokenizer.cls_token_id]
        self.masking_schedule = masking_schedule or []

    def update_policy(self, training_step):
        for step, new_policy in self.masking_schedule:
            if training_step >= step:
                self.policy = new_policy
            else:
                break

    def __call__(self, examples, training_step=None):
        examples_ = examples.clone()
        batch = self._preprocess_batch(examples_)
        input_ids, labels = self.mask_tokens(input_ids=batch, training_step=training_step)
        return input_ids, labels

    def mask_tokens(self, input_ids, training_step=None):
        # Update the masking policy based on the current training step
        if training_step is not None:
            self.update_policy(training_step)
            # print('yedyedy')
        # ... rest of the mask_tokens method remains unchanged ...

        # Preparing masked tokens input and label pairs for MLM
        labels = input_ids.clone()
        probabilities = torch.full(size=labels.shape, fill_value=self.mask_probability)
        special_tokens_mask = []

        for labels_ in labels.tolist():
            mask = self.tokenizer.get_special_tokens_mask(labels_, already_has_special_tokens=True)
            special_tokens_mask.append(mask)

        special_indices = torch.tensor(data=special_tokens_mask, dtype=torch.bool)
        probabilities.masked_fill_(special_indices, value=0.0)

        for special_token_id in self.special_token_ids:
            if special_token_id:
                special_indices = labels.eq(special_token_id)
                probabilities.masked_fill_(mask=special_indices, value=0.0)

        masked_indices = torch.bernoulli(probabilities).bool()
        labels[~masked_indices] = self.nomask_id

        replaced_indices = torch.bernoulli(torch.full(size=labels.shape, fill_value=self.policy[0])).bool()
        replaced_indices = replaced_indices & masked_indices
        input_ids[replaced_indices] = self.mask_id

        randomized_indices = torch.bernoulli(torch.full(size=labels.shape, fill_value=self.policy[1]/(1-self.policy[0]))).bool()
        randomized_indices = randomized_indices & masked_indices & ~replaced_indices
        random_tokens = torch.randint(high=len(self.tokenizer), size=labels.shape, dtype=torch.long)

        allowed_token_ids = [i for i in range(len(self.tokenizer)) if i not in self.special_token_ids]
        for special_token_id in self.special_token_ids:
            replacement_token_id = np.random.choice(allowed_token_ids, size=1)
            random_tokens[random_tokens == special_token_id] = torch.tensor(replacement_token_id, dtype=torch.long)

        input_ids[randomized_indices] = random_tokens[randomized_indices]

        return input_ids, labels


In [ ]:
TOKENIZER = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True,
                                          do_basic_tokenize=True,
                                          unk_token='[UNK]',
                                          sep_token='[SEP]',
                                          pad_token='[PAD]',
                                          cls_token='[CLS]',
                                          mask_token='[MASK]')

encode = partial(TOKENIZER.encode, max_length=48)
pad_token_id = TOKENIZER.convert_tokens_to_ids(TOKENIZER.pad_token)
# NOTE: We add the pad token ID as discussed in https://github.com/pytorch/text/issues/609
TEXT = Field(use_vocab=False, tokenize=encode, pad_token=pad_token_id, batch_first=True)
LABEL = LabelField()

In [ ]:
train_dataset, test_dataset = IMDB.splits(text_field=TEXT, label_field=LABEL)
LABEL.build_vocab(train_dataset)
train_loader, test_loader = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_size=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
def simplify_label(label):
    tokens = label.split()
    tokens = [token for token in tokens if token != '[UNK]']
    tokens = ['%d. %s' % (i + 1, token) for i, token in enumerate(tokens)]
    return ', '.join(tokens)

# MLM

In [ ]:
masking_schedule = [
    (10000, [0.6, 0.3, 0.1])
]

collator = MLMCollator(tokenizer=TOKENIZER, mask_probability=0.25, masking_schedule=masking_schedule)
# collator = MLMCollator(tokenizer=TOKENIZER, mask_probability=0.25)

for batch in train_loader:
    x, _ = batch.text, batch.label
    examples, labels = collator(x)
    print('Input:\n------\n%s \n\nOutput:\n-------\n%s' %
          (TOKENIZER.decode(examples[0].tolist()), simplify_label(TOKENIZER.decode(labels[0].tolist()))))
    break

Input:
------
[CLS] [MASK] was prc thirteen [MASK] wrexham movie [MASK] [MASK] bosch television. it is far superior in action than most movies since. martin sheen is excellent, and though nick nolte has a small part, he too provides excellent support [MASK] vic [SEP] 

Output:
-------
1. i, 2. about, 3. when, 4. this, 5. came, 6. out, 7. on, 8. excellent, 9. [UNK].
